# Automated Audio Transcription of Earnings Call with  Summary, Key Information Extraction and Sentiment Analysis Using LLMs


## Objective 


The following code uses Whisper and GPT-4o to transcribe audio from a meeting, provides a summary of the discussion, extracts key points and action items, and performs a sentiment analysis.

## Install Packages 

Install `python-docx` and `OpenAI` libraries:

In [1]:
#! pip install openai
#! pip install python-docx

## Set up OPEN AI Connect Credentials




In [3]:
from openai import OpenAI

client = OpenAI(api_key = "sk-proj-xxx")



## Transcribe a Given Audio File Using  Whisper Model 

Here we define a function to transcribean audio file. 
When this function is called with the path to an audio file (like a .mp3 or .wav file), it opens the file, sends it to OpenAI's Whisper model using an API request, and returns the transcription of the audio as text.

In [4]:
from docx import Document

def transcribe_audio(audio_file_path):
    with open(audio_file_path, 'rb') as audio_file:
        transcription = client.audio.transcriptions.create(model = "whisper-1", file = audio_file)
    return transcription.text

## Summarize and Analyse Transcribed Audio Using  `GPT-4o` 

Here we pass the text of the transcribed audio to `GPT-4o` to perform the following the tasks 

- summarize the entire discussion 
- extract key points 
- identify action items mentioned in the call
- sentiment analysis 

The meeting_minutes() function as defined below calls four individual user defined functions to achieve the above tasks. 

In [5]:
def meeting_minutes(transcription):
    abstract_summary = abstract_summary_extraction(transcription)
    key_points = key_points_extraction(transcription)
    action_items = action_item_extraction(transcription)
    sentiment = sentiment_analysis(transcription)
    return {
        'abstract_summary': abstract_summary,
        'key_points': key_points,
        'action_items': action_items,
        'sentiment': sentiment
    }

### Summary Extraction 

`abstract_summary_extraction` summarizes the transcription into a concise abstract paragraph with the aim to retain the most important points while avoiding unnecessary details or tangential points. 

In [6]:
def abstract_summary_extraction(transcription):
    completion =  client.chat.completions.create(
        model = "gpt-4o",
        temperature = 0,
        messages=[
            {
                "role": "system",
                "content": "You are a highly skilled AI trained in language comprehension and summarization. I would like you to read the following text and summarize it into a concise abstract paragraph. Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text. Please avoid unnecessary details or tangential points."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return completion.choices[0].message.content

### Key points extraction 

The `key_points_extraction` function identifies and lists the main points discussed in the meeting. These points represent the most important ideas, findings, or topics crucial to the essence of the discussion. 

In [7]:

def key_points_extraction(transcription):
    completion = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a proficient AI with a specialty in distilling information into key points. Based on the following text, identify and list the main points that were discussed or brought up. These should be the most important ideas, findings, or topics that are crucial to the essence of the discussion. Your goal is to provide a list that someone could read to quickly understand what was talked about."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return completion.choices[0].message.content

### Action item extraction 

The `action_item_extraction` function identifies tasks, assignments, or actions agreed upon or mentioned during the meeting.

In [8]:

def action_item_extraction(transcription):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are an AI expert in analyzing conversations and extracting action items. Please review the text and identify any tasks, assignments, or actions that were agreed upon or mentioned as needing to be done. These could be tasks assigned to specific individuals, or general actions that the group has decided to take. Please list these action items clearly and concisely."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return completion.choices[0].message.content

### Sentiment_analysis

The `sentiment_analysis` function analyzes the overall sentiment of the discussion. It considers the tone, the emotions conveyed by the language used, and the context in which words and phrases are used.

In [9]:
def sentiment_analysis(transcription):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "As an AI with expertise in language and emotion analysis, your task is to analyze the sentiment of the following text. Please consider the overall tone of the discussion, the emotion conveyed by the language used, and the context in which words and phrases are used. Indicate whether the sentiment is generally positive, negative, or neutral, and provide brief explanations for your analysis where possible."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return completion.choices[0].message.content

### Export extracted summaries to Word Doc format 

We export the meeting minutes to a Word document. separated by sections 

In [12]:
def save_as_docx(minutes, filename):
    doc = Document()
    for key, value in minutes.items():
        # Replace underscores with spaces and capitalize each word for the heading
        heading = ' '.join(word.capitalize() for word in key.split('_'))
        doc.add_heading(heading, level=1)
        doc.add_paragraph(value)
        # Add a line break between sections
        doc.add_paragraph()
    doc.save(filename)

### Putting it all together 

Finally, we put it all together and generate the meeting minutes from an audio file:

#### Transcribe Audio File 

In [ ]:
audio_file_path = "EarningsCall_processed.wav"
transcription = transcribe_audio(audio_file_path)

#### Extract summaries from transcription 

In [14]:
minutes = meeting_minutes(transcription)
print(minutes)

{'abstract_summary': "FinTech Plus reported a strong second quarter in 2023, with a 25% year-over-year revenue increase to $125 million and a gross profit margin of 58%. The company's EBITDA reached $37.5 million, reflecting a 30% margin, while net income rose to $16 million from $10 million in the previous year. The expansion of their high-yield savings products and a new RoboAdvisor platform has significantly grown their market. They have diversified their asset-backed securities portfolio and invested $25 million in AAA-rated corporate bonds. The balance sheet shows total assets of $1.5 billion and liabilities of $900 million, resulting in $600 million in equity and a debt-to-equity ratio of 1.5. Customer acquisition costs have decreased by 15%, and lifetime value has increased by 25%, with an LTVCAC ratio of 3.5. Risk management includes a value-at-risk model with a 99% confidence level, and a tier-one capital ratio of 12.5%. The company forecasts an 8% quarter-over-quarter revenue

#### Save in word format 

In [15]:
save_as_docx(minutes, 'meeting_minutes.docx')


### End of Notebook